In [86]:
#!pip install numpy requests nlpaug
#!pip install tensformers
import pandas as pd
import numpy as np
import nlpaug.augmenter.word as nlpaw
from tqdm import tqdm
from transformers import BertForSequenceClassification, AdamW, BertConfig,get_linear_schedule_with_warmup
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
# Note: this notebook requires torch >= 1.10.0
torch.__version__
device = "cuda" if torch.cuda.is_available() else "cpu"
device
from transformers import AutoTokenizer


import os
import shutil
# !pip install tensorboard
!pip install accelerate -U
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

In [87]:


from torch.utils.tensorboard import SummaryWriter

TBOARD_LOGS = os.path.join(os.getcwd(), "tboard_logs", "test")
if not os.path.exists(TBOARD_LOGS):
    os.makedirs(TBOARD_LOGS)

shutil.rmtree(TBOARD_LOGS)
writer = SummaryWriter(TBOARD_LOGS)



In [88]:
# !pip uninstall pillow
# !pip install pillow==9.4.0


df=pd.read_csv('../train-balanced-sarcasm.csv')
labels=df["label"].to_list()
comments=df["comment"].astype(str).to_list()
df2=pd.DataFrame({"comments":comments,"labels":labels})
df2_short={"text":comments,"labels":labels}

In [89]:
from sklearn.model_selection import train_test_split
x_train, x_test_valid, y_train, y_test_valid = train_test_split(comments, labels, test_size=0.33, random_state=42)

x_test, x_valid, y_test, y_valid = train_test_split(x_test_valid, y_test_valid, test_size=0.5, random_state=42)

In [90]:
from transformers import DistilBertTokenizerFast
# tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2, output_attentions = False,
    output_hidden_states = False)

optimizer = torch.optim.AdamW(model.parameters(), 
                              lr = 5e-5,
                              eps = 1e-08
                              )

# Run on GPU
# model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [91]:
# def tokenize(texts):
#     tokenized_texts=[]
#     for text in texts:
#         if type(text)!='str':
#             # print(type(text))
#             text=str(text)
#         token=tokenizer(text,
#         truncation=True,
#         padding=True,
#         max_length=128)
#         tokenized_texts.append(token)

#     return tokenized_texts    

In [92]:
def preprocess_function(sent):
    return  tokenizer.encode_plus(
                        sent,
                        add_special_tokens = True,
                        padding=True,
                        max_length = 256,
                        truncation=True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )

In [93]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self,data,labels):
      self.data=data
      self.labels=labels   
    def __len__(self):
        return len(self.data)    
    def __getitem__(self, idx):
       print(type(self.data[idx]))
       return preprocess_function(self.data[idx]) ,torch.tensor(self.labels[idx])
       return [preprocess_function(d) for d in self.data[idx]],torch.tensor(self.labels[idx])

In [94]:

train_dataset=CustomDataset(x_train,labels=y_train)
test_dataset=CustomDataset(x_test,labels=y_test)

In [95]:
train_dataloader=torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=False)
test_dataloader=torch.utils.data.DataLoader(test_dataset)


In [96]:
batch = next(iter(test_dataloader))
batch

<class 'str'>


[{'input_ids': tensor([[[  101, 27327,  2271,  3504, 10140,   102]]]), 'attention_mask': tensor([[[1, 1, 1, 1, 1, 1]]])},
 tensor([0])]

In [97]:
def train_epoch(model, train_loader, optimizer, criterion, epoch, device):
    """ Training a model for one epoch """
    
    loss_list = []
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
         
        # Forward pass to get output/logits
        outputs = model(images)
         
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())
         
        # Getting gradients w.r.t. parameters
        loss.backward()
         
        # Updating parameters
        optimizer.step()
        
    mean_loss = np.mean(loss_list)
    return mean_loss, loss_list


@torch.no_grad()
def eval_model(model, eval_loader, criterion, device):
    """ Evaluating the model for either validation or test """
    correct = 0
    total = 0
    loss_list = []
    
    for images, labels in eval_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass only to get logits/output
        outputs = model(images)
                 
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())
            
        # Get predictions from the maximum value
        preds = torch.argmax(outputs, dim=1)
        correct += len( torch.where(preds==labels)[0] )
        total += len(labels)
                 
    # Total correct predictions and loss
    accuracy = correct / total * 100
    loss = np.mean(loss_list)
    
    return accuracy, loss


def train_model(model, optimizer, scheduler, criterion, train_loader, valid_loader, num_epochs, tboard=None, start_epoch=0):
    """ Training a model for a given number of epochs"""
    
    train_loss = []
    val_loss =  []
    loss_iters = []
    valid_acc = []
    
    for epoch in range(num_epochs):
           
        # validation epoch
        model.eval()  # important for dropout and batch norms
        accuracy, loss = eval_model(
                    model=model, eval_loader=valid_loader,
                    criterion=criterion, device=device
            )
        valid_acc.append(accuracy)
        val_loss.append(loss)
        writer.add_scalar(f'Accuracy/Valid', accuracy, global_step=epoch+start_epoch)
        writer.add_scalar(f'Loss/Valid', loss, global_step=epoch+start_epoch)
        
        # training epoch
        model.train()  # important for dropout and batch norms
        mean_loss, cur_loss_iters = train_epoch(
                model=model, train_loader=train_loader, optimizer=optimizer,
                criterion=criterion, epoch=epoch, device=device
            )
        scheduler.step()
        train_loss.append(mean_loss)
        writer.add_scalar(f'Loss/Train', mean_loss, global_step=epoch+start_epoch)

        loss_iters = loss_iters + cur_loss_iters
        
        if(epoch % 5 == 0 or epoch==num_epochs-1):
            print(f"Epoch {epoch+1}/{num_epochs}")
            print(f"    Train loss: {round(mean_loss, 5)}")
            print(f"    Valid loss: {round(loss, 5)}")
            print(f"    Accuracy: {accuracy}%")
            print("\n")
    
    print(f"Training completed")
    return train_loss, val_loss, loss_iters, valid_acc

In [98]:
!pip install evaluate
import evaluate

accuracy = evaluate.load("accuracy")

In [99]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [100]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    # data_collator=data_collator,
    compute_metrics=compute_metrics,
    
)

trainer.train()



ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`